In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [5]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [6]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [7]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [8]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [9]:
# load link counts data

import json

with open('../temp_files/link_day_minute_Jul_dict_JSON.json', 'r') as json_file:
    link_day_minute_Jul_dict_JSON = json.load(json_file)

In [18]:
#week_day_Jul_list = [2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 30, 31]
week_day_Jul_list = [2, 9, 16, 23, 30]

In [19]:
link_day_minute_Jul_list = []
for link_idx in range(24):
    for day in week_day_Jul_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Jul_list.append(link_day_minute_Jul_dict_JSON[key] ['MD_flow_minute'][minute_idx])

In [20]:
len(link_day_minute_Jul_list)

14400

In [21]:
x = np.matrix(link_day_minute_Jul_list)
x = np.matrix.reshape(x, 24, 600)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [22]:
np.size(x, 0), np.size(x, 1)

(24, 600)

In [23]:
x[:,:2]

matrix([[ 2370.70334479,  2360.58464542],
        [ 2301.68959261,  2316.59146776],
        [ 2889.56175804,  2851.70977554],
        [ 2802.73660851,  2848.86924585],
        [ 2890.27324062,  2909.09044506],
        [ 2762.04741028,  2702.96620738],
        [ 2432.38992652,  2455.0054697 ],
        [ 2397.31898105,  2350.4609361 ],
        [ 3207.72162926,  3233.60089579],
        [ 3161.88766542,  3165.86712397],
        [ 2667.18422786,  2756.84253243],
        [ 2346.15032178,  2398.83749558],
        [ 2305.05155978,  2324.44571121],
        [ 2603.74239828,  2527.28916418],
        [ 2323.60918916,  2414.09115152],
        [ 1890.51064006,  1808.37206144],
        [ 2349.79749197,  2404.79056022],
        [ 2373.61518481,  2339.44023135],
        [ 2618.94647118,  2346.56745078],
        [ 2749.7849551 ,  2688.51560092],
        [ 2940.66447176,  2953.40415411],
        [ 2439.54837863,  2628.20302865],
        [ 2266.44866872,  2385.61203068],
        [ 2303.77086466,  2390.501

In [24]:
np.size(A,0), np.size(A,1)

(24, 140)

In [25]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
print('Obj: %g' % obj.getValue())

Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [2e+01, 4e+03]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.00s
Presolved: 0 rows, 56 columns, 0 nonzeros
Presolved model has 1596 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 26
 AA' NZ     : 3.250e+02
 Factor NZ  : 3.510e+02
 Factor Ops : 6.201e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -7.47922008e+07  0.00000000e+00  9.57e+03 2.44e+03  1.00e+06     0s
   1  -2.80341241e+07 -1.83549455e+07  1.30e+03 3.32e+02  2.23e+05     0s
   2  -1.86436940e+07 -1.95772259e+07  3.77e+01 9.61e+00  2.71e+04     0s
   3  -1.84490907e+07 -1.85593684e+07  2.48e+00 6.33e-01  2.6

In [26]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix_Jul_weekday_MD.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [27]:
lam_list

[1.935110519391285e-10,
 4630.789913900822,
 2.1241136922231808e-10,
 0.031274744459028364,
 0.12856347645796973,
 0.691266610124905,
 6.4099161282207e-08,
 1.7642784994666036e-05,
 181.40500428941283,
 2498.8444469175492,
 0.04918420404523355,
 0.4047511769685155,
 0.620332579852187,
 20.003525725124554,
 3776.014796899414,
 923.4669517092343,
 382.37462419564804,
 17.580935325677785,
 898.8093378413696,
 4315.363094203159,
 0.7242970923897554,
 1381.7883911020488,
 1172.9871190187566,
 0.09910157738448011,
 2020.5745936809558,
 48.80107077839149,
 5.543720510791067e-05,
 2143.1567191089857,
 0.13522939493710395,
 2.0584727126806968,
 382.22715544141704,
 0.40252284512806114,
 2311.3937547596356,
 5.48871030088651e-05,
 5.491344828623807e-05,
 9.84843977961778e-07,
 9.84841870269109e-07,
 9.848411120835974e-07,
 1936.6049119233082,
 0.12828185849064466,
 494.792454141546,
 492.3932652201356,
 1.1132837843031602,
 748.9210908299343,
 1946.4403632691713,
 86.22095172794864,
 0.509875970